In [1]:
import pandas as pd

In [2]:
visitation_data = "visitation_d.csv"
climate_data = "climate_d.csv"

visit_df = pd.read_csv(visitation_data)
clim_df = pd.read_csv(climate_data)

## Cleaning visitation data

In [3]:
visit_df.head(5)

,Year,Week,Mt. Baw Baw,Mt. Stirling,Mt. Hotham,Falls Creek,Mt. Buller,Selwyn,Thredbo,Perisher,Charlotte Pass
0,2014.0,1.0,555.0,60.0,3483.0,2790.0,8296.0,1041.0,5535.0,7370.0,408.0
1,2014.0,2.0,804.0,42.0,1253.0,1425.0,1987.0,383.0,2090.0,2751.0,151.0
2,2014.0,3.0,993.0,30.0,2992.0,2101.0,2413.0,597.0,3216.0,4255.0,230.0
3,2014.0,4.0,2976.0,165.0,9680.0,9544.0,18831.0,2877.0,15497.0,20265.0,1134.0
4,2014.0,5.0,11112.0,645.0,29628.0,26211.0,49217.0,8588.0,46546.0,61339.0,3403.0


In [4]:
visit_df.dtypes

Year              float64
Week              float64
Mt. Baw Baw       float64
Mt. Stirling      float64
Mt. Hotham        float64
Falls Creek       float64
Mt. Buller        float64
Selwyn            float64
Thredbo           float64
Perisher          float64
Charlotte Pass    float64
dtype: object

In [5]:
visit_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Year            165 non-null    float64
 1   Week            165 non-null    float64
 2   Mt. Baw Baw     165 non-null    float64
 3   Mt. Stirling    165 non-null    float64
 4   Mt. Hotham      165 non-null    float64
 5   Falls Creek     165 non-null    float64
 6   Mt. Buller      165 non-null    float64
 7   Selwyn          165 non-null    float64
 8   Thredbo         165 non-null    float64
 9   Perisher        165 non-null    float64
 10  Charlotte Pass  165 non-null    float64
dtypes: float64(11)
memory usage: 86.0 KB


In [6]:
visit_df.count() # check for null values

Year              165
Week              165
Mt. Baw Baw       165
Mt. Stirling      165
Mt. Hotham        165
Falls Creek       165
Mt. Buller        165
Selwyn            165
Thredbo           165
Perisher          165
Charlotte Pass    165
dtype: int64

In [7]:
visit_df.describe()

,Year,Week,Mt. Baw Baw,Mt. Stirling,Mt. Hotham,Falls Creek,Mt. Buller,Selwyn,Thredbo,Perisher,Charlotte Pass
count,165.000000,165.000000,165.000000,165.000000,165.000000,165.000000,165.000000,165.000000,165.000000,165.000000,165.000000
mean,2019.000000,8.000000,5373.648485,589.981818,20110.333333,24027.145455,29349.296970,5536.533333,30020.884848,39649.515152,2181.812121
std,3.171904,4.333646,3924.269031,529.497558,12198.935589,14176.688461,17768.711945,3383.338655,18363.819945,24258.783674,1334.790842
min,2014.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2016.000000,4.000000,1757.000000,98.000000,8368.000000,8741.000000,12397.000000,2180.000000,11543.000000,15236.000000,842.000000
50%,2019.000000,8.000000,5565.000000,480.000000,21856.000000,28099.000000,33328.000000,6454.000000,34676.000000,46265.000000,2542.000000
75%,2022.000000,12.000000,8513.000000,962.000000,30944.000000,36692.000000,44315.000000,8530.000000,46015.000000,60788.000000,3344.000000
max,2024.000000,15.000000,18031.000000,2097.000000,44648.000000,49697.000000,66326.000000,12863.000000,70634.000000,93226.000000,5033.000000


## Cleaning climate data

In [8]:
clim_df.head(5)

,Bureau of Meteorology station number,Year,Month,Day,Maximum temperature (Degree C),Minimum temperature (Degree C),Rainfall amount (millimetres)
0,71075,2010,1,1,NaN,NaN,NaN
1,71075,2010,1,2,NaN,NaN,NaN
2,71075,2010,1,3,NaN,NaN,NaN
3,71075,2010,1,4,NaN,NaN,NaN
4,71075,2010,1,5,NaN,NaN,NaN


In [9]:
clim_df.columns

Index(['Bureau of Meteorology station number', 'Year', 'Month', 'Day',
       'Maximum temperature (Degree C)', 'Minimum temperature (Degree C)',
       'Rainfall amount (millimetres)'],
      dtype='object')

In [10]:
clim_df.rename(columns={'Bureau of Meteorology station number': 'mount_id', "Maximum temperature (Degree C)": "max_temp", "Minimum temperature (Degree C)": "min_temp",
"Rainfall amount (millimetres)":"rain"}, inplace=True)

In [11]:
clim_df.columns

Index(['mount_id', 'Year', 'Month', 'Day', 'max_temp', 'min_temp', 'rain'], dtype='object')

In [12]:
clim_df.dropna(how='any', inplace=True)

In [13]:
clim_df.dtypes

mount_id      int64
Year          int64
Month         int64
Day           int64
max_temp    float64
min_temp    float64
rain        float64
dtype: object

In [14]:
clim_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 36820 entries, 145 to 39812
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   mount_id  36820 non-null  int64  
 1   Year      36820 non-null  int64  
 2   Month     36820 non-null  int64  
 3   Day       36820 non-null  int64  
 4   max_temp  36820 non-null  float64
 5   min_temp  36820 non-null  float64
 6   rain      36820 non-null  float64
dtypes: float64(3), int64(4)
memory usage: 2.2 MB


In [15]:
clim_df.describe()

,mount_id,Year,Month,Day,max_temp,min_temp,rain
count,36820.000000,36820.000000,36820.000000,36820.000000,36820.000000,36820.000000,36820.000000
mean,78246.758365,2017.185117,6.402173,15.755024,9.873308,2.726285,4.661847
std,6110.605165,4.491438,3.420067,8.798887,7.098949,5.635775,11.147340
min,71032.000000,2010.000000,1.000000,1.000000,-5.800000,-14.200000,0.000000
25%,71075.000000,2013.000000,3.000000,8.000000,4.000000,-1.700000,0.000000
50%,83024.000000,2017.000000,6.000000,16.000000,9.700000,2.000000,0.000000
75%,83085.000000,2021.000000,9.000000,23.000000,15.500000,7.000000,3.600000
max,85291.000000,2025.000000,12.000000,31.000000,34.000000,22.700000,158.600000


In [16]:
stations = {
    'Thredbo': 71032,
    'Perisher': 71075,
    'Selwyn': 72161,
    'Mt. Buller': 83024,
    'Falls Creek': 83084,
    'Mt. Hotham': 83085,
    'Mt. Baw Baw': 85291
}

In [17]:
for i in visit_df.columns[2:]: # Trying to rename columns using the stations dictionary
    try:
        visit_df.rename(columns={i: stations[i]}, inplace=True)
    except KeyError:
        continue
visit_df.columns

Index([          'Year',           'Week',            85291,   'Mt. Stirling',
                  83085,            83084,            83024,            72161,
                  71032,            71075, 'Charlotte Pass'],
      dtype='object')

In [18]:
clim_df["mount_id"].unique()

array([71075, 71032, 83024, 83084, 83085, 85291, 72161])

## Removed uneeded things

In [19]:
new_visit_df = visit_df
for i in visit_df.columns[2:]:
    if type(i) != int:
        new_visit_df.drop(columns=i, inplace=True)
new_visit_df.columns

Index(['Year', 'Week', 85291, 83085, 83084, 83024, 72161, 71032, 71075], dtype='object')

In [20]:
new_clim_df = clim_df.drop(
    clim_df[(clim_df["Year"] < 2014) | (clim_df["Year"] > 2024)].index
)

In [21]:
new_clim_df.describe() #checking if the filtering worked

,mount_id,Year,Month,Day,max_temp,min_temp,rain
count,25790.000000,25790.000000,25790.000000,25790.000000,25790.000000,25790.000000,25790.000000
mean,78099.514812,2018.920163,6.452385,15.772509,10.018957,2.814564,4.549035
std,6123.653719,3.201098,3.427184,8.799123,7.118285,5.641051,10.623513
min,71032.000000,2014.000000,1.000000,1.000000,-5.800000,-14.200000,0.000000
25%,71075.000000,2016.000000,3.000000,8.000000,4.100000,-1.500000,0.000000
50%,83024.000000,2019.000000,6.000000,16.000000,9.800000,2.100000,0.000000
75%,83085.000000,2022.000000,9.000000,23.000000,15.700000,7.000000,3.600000
max,85291.000000,2024.000000,12.000000,31.000000,34.000000,22.700000,158.600000


In [22]:
# Convert Month and Day to a datetime, then get week number
new_clim_df['date'] = pd.to_datetime(dict(year=new_clim_df['Year'], month=new_clim_df['Month'], day=new_clim_df['Day']))
new_clim_df['week'] = new_clim_df['date'].dt.isocalendar().week

# Filter for weeks 1 to 15
new_clim_df = new_clim_df[(new_clim_df['week'] >= 1) & (new_clim_df['week'] <= 15)]

# Group by Year and week, take average of other columns
new_clim_df = new_clim_df.groupby(['mount_id','Year', 'week'], as_index=False).mean(numeric_only=True)

In [23]:
new_clim_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1111 entries, 0 to 1110
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   mount_id  1111 non-null   int64  
 1   Year      1111 non-null   int64  
 2   week      1111 non-null   UInt32 
 3   Month     1111 non-null   float64
 4   Day       1111 non-null   float64
 5   max_temp  1111 non-null   float64
 6   min_temp  1111 non-null   float64
 7   rain      1111 non-null   float64
dtypes: UInt32(1), float64(5), int64(2)
memory usage: 66.3 KB


In [24]:
new_clim_df.drop(columns=['Month', 'Day'], inplace=True)

In [25]:
new_clim_df.columns

Index(['mount_id', 'Year', 'week', 'max_temp', 'min_temp', 'rain'], dtype='object')

In [26]:
new_clim_df = new_clim_df[['mount_id', 'Year', 'week',  'max_temp', 'min_temp', 'rain']]

In [27]:
new_clim_df.to_csv("cleaned_climate_data.csv", index=False)

## Removing outlier

In [28]:
new_visit_df.describe()

,Year,Week,85291,83085,83084,83024,72161,71032,71075
count,165.000000,165.000000,165.000000,165.000000,165.000000,165.000000,165.000000,165.000000,165.000000
mean,2019.000000,8.000000,5373.648485,20110.333333,24027.145455,29349.296970,5536.533333,30020.884848,39649.515152
std,3.171904,4.333646,3924.269031,12198.935589,14176.688461,17768.711945,3383.338655,18363.819945,24258.783674
min,2014.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2016.000000,4.000000,1757.000000,8368.000000,8741.000000,12397.000000,2180.000000,11543.000000,15236.000000
50%,2019.000000,8.000000,5565.000000,21856.000000,28099.000000,33328.000000,6454.000000,34676.000000,46265.000000
75%,2022.000000,12.000000,8513.000000,30944.000000,36692.000000,44315.000000,8530.000000,46015.000000,60788.000000
max,2024.000000,15.000000,18031.000000,44648.000000,49697.000000,66326.000000,12863.000000,70634.000000,93226.000000


In [29]:
from sklearn.ensemble import IsolationForest

mountain_cols = [c for c in new_visit_df.columns if c not in ['Year','Week']]
outlier_indices = set()
mult = 1.5

X = new_visit_df[mountain_cols].apply(pd.to_numeric, errors='coerce')

# simple imputation for missing values — median is robust
X_imp = X.fillna(X.median())

clf = IsolationForest(contamination=0.01, random_state=0)   # adjust contamination
clf.fit(X_imp)
new_visit_df['iso_outlier'] = clf.predict(X_imp) == -1

new_visit_df = new_visit_df.loc[~new_visit_df['iso_outlier']].reset_index(drop=True)

In [30]:
new_visit_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 989 entries, 0 to 988
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Year         155 non-null    float64
 1   Week         155 non-null    float64
 2   85291        155 non-null    float64
 3   83085        155 non-null    float64
 4   83084        155 non-null    float64
 5   83024        155 non-null    float64
 6   72161        155 non-null    float64
 7   71032        155 non-null    float64
 8   71075        155 non-null    float64
 9   iso_outlier  989 non-null    bool   
dtypes: bool(1), float64(9)
memory usage: 70.6 KB


In [31]:
new_visit_df.drop(columns=['iso_outlier'], inplace=True)

In [32]:
new_visit_df.describe()

,Year,Week,85291,83085,83084,83024,72161,71032,71075
count,155.00000,155.000000,155.000000,155.000000,155.000000,155.000000,155.000000,155.000000,155.000000
mean,2019.00000,7.877419,5308.954839,20197.122581,24353.490323,29322.845161,5532.561290,29985.896774,39614.109677
std,3.22933,4.348959,3660.760771,11631.934082,13735.225181,16654.311432,3183.141757,17254.416567,22812.514037
min,2014.00000,1.000000,0.000000,0.000000,1425.000000,0.000000,311.000000,1666.000000,2205.000000
25%,2016.00000,4.000000,2003.000000,9117.000000,9920.500000,13177.000000,2550.000000,13621.000000,17890.000000
50%,2019.00000,8.000000,5565.000000,21856.000000,28099.000000,33328.000000,6454.000000,34676.000000,46265.000000
75%,2022.00000,12.000000,8385.500000,30634.500000,36200.500000,44053.500000,8430.000000,45834.000000,60294.000000
max,2024.00000,15.000000,17116.000000,44648.000000,49697.000000,58192.000000,10213.000000,55420.000000,72800.000000


In [33]:
col_to_check = [c for c in new_clim_df.columns if c not in ['mount_id','Year','week']]
print(len(new_clim_df))

1111


In [34]:
outlier_indices = set()
mult = 1.5

X = new_clim_df[col_to_check].apply(pd.to_numeric, errors='coerce')

# simple imputation for missing values — median is robust
X_imp = X.fillna(X.median())

clf = IsolationForest(contamination=0.01, random_state=0)   # adjust contamination
clf.fit(X_imp)
new_clim_df['iso_outlier'] = clf.predict(X_imp) == -1

new_clim_df = new_clim_df.loc[~new_clim_df['iso_outlier']].reset_index(drop=True)

In [35]:
new_clim_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1099 entries, 0 to 1098
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   mount_id     1099 non-null   int64  
 1   Year         1099 non-null   int64  
 2   week         1099 non-null   UInt32 
 3   max_temp     1099 non-null   float64
 4   min_temp     1099 non-null   float64
 5   rain         1099 non-null   float64
 6   iso_outlier  1099 non-null   bool   
dtypes: UInt32(1), bool(1), float64(3), int64(2)
memory usage: 49.5 KB


In [36]:
new_clim_df.drop(columns=['iso_outlier'], inplace=True)

In [37]:
new_visit_df.to_csv("cleaned_visit_data.csv", index=False)
new_clim_df.to_csv("cleaned_climate_data.csv", index=False)